# OpenAPI Plugin in Semantic Kernel

## Overview

This notebook demonstrates how to integrate external APIs into Semantic Kernel using OpenAPI plugins.

### The Problem

AI agents often need to interact with external services and APIs to provide meaningful, actionable responses. However, manually creating function definitions for every API endpoint is time-consuming and error-prone. Additionally, many enterprise APIs require complex authentication mechanisms that need to be handled securely.

### The Solution

OpenAPI plugins in Semantic Kernel automatically generate function definitions from OpenAPI specifications, enabling seamless integration with external services. This approach provides:

* Automatic Function Generation: Convert OpenAPI specs into callable kernel functions
* Authentication Handling: Built-in support for various authentication methods including Bearer tokens
* Type Safety: Strongly-typed function parameters based on API specifications
* Enterprise Integration: Easy connection to services like Azure DevOps, GitHub, and other REST APIs

## Package References and Configuration

First, let's install the required NuGet packages and create our kernel

In [ ]:
#r "nuget: Microsoft.SemanticKernel, 1.61.0"

#!import config/Settings.cs

using Microsoft.SemanticKernel;
using Kernel = Microsoft.SemanticKernel.Kernel;

Kernel CreateKernel()
{
    var builder = Kernel.CreateBuilder();

    var (useAzureOpenAI, model, azureEndpoint, apiKey, orgId, embeddingEndpoint, embeddingApiKey) = Settings.LoadFromFile();

    builder.AddAzureOpenAIChatCompletion(model, azureEndpoint, apiKey);
    var kernel = builder.Build();

    return kernel;
}

var kernel = CreateKernel();

Installed Packages Microsoft.SemanticKernel, 1.60.0

In [8]:
#r "nuget: Azure.Identity, 1.13.1"
#r "nuget: Azure.Core, 1.44.1"

Installed Packages azure.core, 1.44.1 Azure.Identity, 1.13.1

In [10]:
using Azure.Core;
using Azure.Identity;

TokenCredential tokenCredential =
    new ChainedTokenCredential(
        new AzureCliCredential(),
        new VisualStudioCredential(),
        new DefaultAzureCredential(
            new DefaultAzureCredentialOptions()));

In [4]:
var tokenRequestContext = new TokenRequestContext(["499b84ac-1321-427f-aa17-267ca6975798/.default"], parentRequestId: null);
var accesstoken = await tokenCredential.GetTokenAsync(tokenRequestContext, System.Threading.CancellationToken.None);

In [5]:
#r "nuget: Microsoft.SemanticKernel.Plugins.OpenApi.Extensions, 1.60.0-alpha"

Installed Packages Microsoft.SemanticKernel.Plugins.OpenApi.Extensions, 1.60.0-alpha

In [7]:
using Microsoft.SemanticKernel.Plugins.OpenApi;
using System.Net.Http.Headers;
#pragma warning disable SKEXP0040

KernelPlugin plugin = await OpenApiKernelPluginFactory.CreateFromOpenApiAsync(
    pluginName: "ADOPipelines",     
    filePath: "ado_pipelines_openapi.json",
    new OpenApiFunctionExecutionParameters()
    {
        ServerUrlOverride = new Uri("https://dev.azure.com/"),
        AuthCallback = (request, token) =>
        {
            request.Headers.Authorization = new AuthenticationHeaderValue("Bearer", accesstoken.Token);
            Console.WriteLine($"Requesting {request.RequestUri} ");
            return Task.CompletedTask;
        } 
    }
);
kernel.Plugins.Add(plugin);

In [9]:
PromptExecutionSettings settings = new() { FunctionChoiceBehavior = FunctionChoiceBehavior.Auto() };
var result = await kernel.InvokePromptAsync("give me the 3 first names of my ADO pipelines in my organization MY_ORGANIZATION in project MY_PROJECT api version 7.0", new(settings));
Console.WriteLine(result);

Requesting https://dev.azure.com/MY_ORGANIZATION/MY_PROJECT/_apis/pipelines?$top=3&api-version=7.0 
It seems that the organization, project, or API version details provided might be incorrect, or the organization/project doesn't exist in Azure DevOps. Please verify the details and ensure that they are valid.
